<a href="https://colab.research.google.com/github/2359181042/GNN_cs224w/blob/main/Wells_GNN_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric
!pip install pytorch_lightning

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 1.4 MB 6.1 MB/s 
     |████████████████████████████████| 222 kB 5.3 MB/s 
     |████████████████████████████████| 376 kB 9.4 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
     |████████████████████████████████| 813 kB 5.4 MB/s 
     |████████████████████████████████| 636 kB 18.8 MB/s 
     |████████████████████████████████| 10.6 MB 2.4 MB/s 
     |████████████████████████████████| 234 kB 46.0 MB/s 
     |████████████████████████████████| 118 kB 48.7 MB/s 
     |████████████████████████████████| 829 kB 56.2 MB/s 
     |████████████████████████████████| 1.3 MB 46.3 MB/s 
     |████████████████████████████████| 142 kB 73.0 MB/s 
     |████████████████████████████████| 294 kB 46.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=3f93a8d483b171e1668e64a087c96004368848a6fd38f4fd4a8def464d6c1b1c
  Stored in directory: /root/.cache/pip/whe

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import os
import json
import numpy as np
import pandas as pd
DATA_PATH = 'gdrive/My Drive/DaguRiver/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/gdrive


In [2]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import math
import matplotlib

import pandas as pd
import numpy as np
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid',palette='muted',font_scale = 1.2)
HAPPY_COLORS_PALETTE = ['#01BEFE','#FFDD00','#FF7D00','#FF006D','#ADFF02','#8F00FF']
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize']=12,8
tqdm.pandas()


ModuleNotFoundError: ignored

# 1. Data Processing
## Totally 10 features: salinity, level, pump, infil, infil_s, rain,loc, duration, well_conn, well_weight.
## sequence length is 18, features salinity-rain is based on 11 opration wells, while feaures loc is based on 1 observe well. The well_conn and well weight are calculated based on the relative distance between two wells.

## 1.1 Prepare data

In [81]:
test1=pd.read_csv(os.path.join(DATA_PATH,'df_of_well.csv'))
salinity=[]
level=[]
pump=[]
infil=[]
infil_s=[]
rain=[]
for i in range(21):
    salinity.append(np.array(test1.iloc[:,0+i*6]))

    level.append(np.array(test1.iloc[:,1+i*6]))

    pump.append(np.absolute(np.array(test1.iloc[:,2+i*6])))

    infil.append(np.array(test1.iloc[:,3+i*6]))

    infil_s.append(np.array(test1.iloc[:,4+i*6]))

    rain.append(np.array(test1.iloc[:,5+i*6]))
salinity=np.array(salinity)
level=np.array(level)
pump = np.array(pump)
infil = np.array(infil)
infil_s = np.array(infil_s)
rain=np.array(rain)            
print(pump[11])

[ 0.   38.3  38.3  38.3  38.3  38.3  38.3  38.3  37.   37.   37.   37.
 37.   38.   38.   38.   38.   38.   35.86 35.86 35.86 35.86 35.86 35.86
 39.86 39.86 39.86 39.86 38.12 38.12 38.12 35.66 35.66 35.66 35.66 35.01
 35.01 35.01 36.96 36.96 36.96 36.96 35.66 35.66 35.66 35.66 35.66 35.66
  0.    0.   37.   38.3  38.3  38.3  38.3  35.01 35.01 35.01 35.01 35.66
 35.66 35.66 35.66 35.01 35.01 35.01 33.13 33.13 33.13 32.9  32.9  32.9
 33.13 33.13 33.13 33.13 33.13 33.13 31.3  31.3  31.3 ]


In [4]:
df3=pd.read_csv(os.path.join(DATA_PATH,'date_info.csv'))
df3['Date'] = pd.to_datetime(df3['Date'])

df3['Day'] = df3['Date'].dt.day
df3['Month']=df3['Date'].dt.month


df3['Duration'] = df3['Day']-17 + 30*(df3['Month']-9) 
duration = df3['Duration'].to_numpy()

In [41]:
Well_location= pd.read_csv(os.path.join(DATA_PATH,'sets.csv'))

NE=[]
North=[]
East=[]
for i in range(1,22):
    N=Well_location.iloc[0][i]
    North.append(N)
    E=Well_location.iloc[1][i]
    East.append(E)
    NE.append((N,E))

In [6]:
from math import sin, cos, sqrt, atan2, radians
R = 6378.1370
def get_conn_attr(N,E):
    Distance = []
    con_i=[]
    con_j=[]
    for i in range(len(N)):
        for j in range(len(N)):
                lat1 = radians(N[i])
                lon1 = radians(E[i])
                lat2 = radians(N[j])
                lon2 = radians(E[j])
                dlon = lon2 - lon1
                dlat = lat2 - lat1
                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))
                distance = R * c
                if (distance < 0.1) & (distance > 0):
                    con_i.append(i)
                    con_j.append(j)
                    Distance.append(distance)

    conn=torch.tensor([con_i,con_j])
    edge_attr = torch.tensor([Distance])
    edge_attr = edge_attr.transpose(0, 1) 
    edge_attr = edge_attr.unsqueeze(0).repeat(18,1,1)
    return conn, edge_attr

In [50]:
list1=NE[10:21]
list2=[NE[0]]
NE1=list1+list2
print(len(NE1))
list3=[NE[1]]
print(list1+list3)





12
[(36.361608000000004, 120.17305900000001), (36.360627, 120.17312), (36.359646000000005, 120.17381699999999), (36.357737, 120.173676), (36.35668, 120.170804), (36.359432, 120.176148), (36.358961, 120.176022), (36.358132, 120.175868), (36.356833, 120.17463400000001), (36.356055, 120.173933), (36.361231, 120.17415), (36.3612222, 120.170423)]


In [64]:
salinity[[10,11,12,13,14,15,16,17,18,19,20,1],80:80+TIME_WINDOW]

array([[0.83 ],
       [0.9  ],
       [0.92 ],
       [  nan],
       [0.76 ],
       [0.92 ],
       [0.94 ],
       [0.91 ],
       [0.84 ],
       [0.82 ],
       [0.87 ],
       [0.865]])

## 1.2 Create dataset\

In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import os
import json
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

TIME_WINDOW = 18
PRED_TIME = 18
##well o 2 4 5  6  8 10 for train_set, 3 9 for validation set, 7 11 for test set
## North[]. East[] 21 用于传入get_conn_attr 计算不同的图数据
 

def create_dataset(salinity,level,pump,infil,infil_s,rain,NE,duration):
    index=0
    train_sequences=[]
    val_sequences=[]
    test_sequences=[]
#O2
    while (index <= (81-PRED_TIME-TIME_WINDOW)):

        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,0],index:index+TIME_WINDOW]
        print(len(s))
        l=level[[10,11,12,13,14,15,16,17,18,19,20,0],index:index+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,0],index:index+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,0],index:index+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,0],index:index+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,0],index:index+TIME_WINDOW]
        N0= North[10:21]+[North[0]]
        E0= East[10:21]+[East[0]]
        conn,edge_attr = get_conn_attr(N0,E0)
        list1=NE[10:21]
        list2=[NE[0]]
        loc=list1+list2
        print(len(loc))
        t=duration[index]
        y = salinity[0,index+TIME_WINDOW:index+TIME_WINDOW+PRED_TIME]
        
        
        
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1
    while ((81-PRED_TIME-TIME_WINDOW)<index<81):
        index +=1

        
#O3        
    while ( 81<=index<=(2*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,1],index:index-1*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,1],index:index-1*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,1],index:index-1*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,1],index:index-1*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,1],index:index-1*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,1],index:index-1*81+TIME_WINDOW]
        N1= North[10:21]+[North[1]]
        E1= East[10:21]+[East[1]]
        conn,edge_attr = get_conn_attr(N1,E1)
        list1=NE[10:21]
        list2=[NE[1]]
        loc=list1+list2
        t=duration[index-81]
        y = salinity[1,index-1*81+TIME_WINDOW:index-1*81+TIME_WINDOW+PRED_TIME]
        val_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((2*81-PRED_TIME-TIME_WINDOW)<index<2*81):
        index +=1    

#O4
    while ( 2*81<=index<=(3*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,2],index-2*81:index-2*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,2],index-2*81:index-2*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,2],index-2*81:index-2*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,2],index-2*81:index-2*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,2],index-2*81:index-2*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,2],index-2*81:index-2*81+TIME_WINDOW]
        N2= North[10:21]+[North[2]]
        E2= East[10:21]+[East[2]]
        conn,edge_attr = get_conn_attr(N2,E2)
        list1=NE[10:21]
        list2=[NE[2]]
        loc=list1+list2
        t=duration[index-81*2]
        y = salinity[2,index-2*81+TIME_WINDOW:index-2*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((3*81-PRED_TIME-TIME_WINDOW)<index<3*81):
        index +=1     

#O5 print('O4finish')      
    while ( 3*81<=index<=(4*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,3],index-3*81:index-3*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,3],index-3*81:index-3*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,3],index-3*81:index-3*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,3],index-3*81:index-3*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,3],index-3*81:index-3*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,3],index-3*81:index-3*81+TIME_WINDOW]
        N3= North[10:21]+[North[3]]
        E3= East[10:21]+[East[3]]
        conn,edge_attr = get_conn_attr(N3,E3)
        list1=NE[10:21]
        list2=[NE[3]]
        loc=list1+list2
        t=duration[index-81*3]
        y = salinity[3,index-3*81+TIME_WINDOW:index-3*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((4*81-PRED_TIME-TIME_WINDOW)<index<4*81):
        index +=1           
#O6       
    while ( 4*81<=index<=(5*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,4],index-4*81:index-4*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,4],index-4*81:index-4*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,4],index-4*81:index-4*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,4],index-4*81:index-4*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,4],index-4*81:index-4*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,4],index-4*81:index-4*81+TIME_WINDOW]
        N4= North[10:21]+[North[4]]
        E4= East[10:21]+[East[4]]
        conn,edge_attr = get_conn_attr(N4,E4)
        list1=NE[10:21]
        list2=[NE[4]]
        loc=list1+list2
        t=duration[index-81*4]
        y = salinity[4,index-4*81+TIME_WINDOW:index-4*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((5*81-PRED_TIME-TIME_WINDOW)<index<5*81):
        index +=1  

#O7       
    while ( 5*81<=index<=(6*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,5],index-5*81:index-5*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,5],index-5*81:index-5*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,5],index-5*81:index-5*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,5],index-5*81:index-5*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,5],index-5*81:index-5*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,5],index-5*81:index-5*81+TIME_WINDOW]
        N5= North[10:21]+[North[5]]
        E5= East[10:21]+[East[5]]
        conn,edge_attr = get_conn_attr(N5,E5)
        list1=NE[10:21]
        list2=[NE[5]]
        loc=list1+list2
        t=duration[index-81*5]
        y = salinity[5,index-5*81+TIME_WINDOW:index-5*81+TIME_WINDOW+PRED_TIME]
        test_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((6*81-PRED_TIME-TIME_WINDOW)<index<6*81):
        index +=1 
#O8      
    while ( 6*81<=index<=(7*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,6],index-6*81:index-6*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,6],index-6*81:index-6*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,6],index-6*81:index-6*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,6],index-6*81:index-6*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,6],index-6*81:index-6*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,6],index-6*81:index-6*81+TIME_WINDOW]
        N6= North[10:21]+[North[6]]
        E6= East[10:21]+[East[6]]
        conn,edge_attr = get_conn_attr(N6,E6)
        list1=NE[10:21]
        list2=[NE[6]]
        loc=list1+list2
        t=duration[index-81*6]
        y = salinity[6,index-6*81+TIME_WINDOW:index-6*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((7*81-PRED_TIME-TIME_WINDOW)<index<7*81):
        index +=1         
#O9        
    while ( 7*81<=index<=(8*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,7],index-7*81:index-7*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,7],index-7*81:index-7*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,7],index-7*81:index-7*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,7],index-7*81:index-7*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,7],index-7*81:index-7*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,7],index-7*81:index-7*81+TIME_WINDOW]
        N7= North[10:21]+[North[7]]
        E7= East[10:21]+[East[7]]
        conn,edge_attr = get_conn_attr(N7,E7)
        list1=NE[10:21]
        list2=[NE[7]]
        loc=list1+list2
        t=duration[index-81*7]
        y = salinity[7,index-7*81+TIME_WINDOW:index-7*81+TIME_WINDOW+PRED_TIME]
        val_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((8*81-PRED_TIME-TIME_WINDOW)<index<8*81):
        index +=1 
#O10        
    while ( 8*81<=index<=(9*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,8],index-8*81:index-8*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,8],index-8*81:index-8*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,8],index-8*81:index-8*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,8],index-8*81:index-8*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,8],index-8*81:index-8*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,8],index-8*81:index-8*81+TIME_WINDOW]
        N8= North[10:21]+[North[8]]
        E8= East[10:21]+[East[8]]
        conn,edge_attr = get_conn_attr(N8,E8)
        list1=NE[10:21]
        list2=[NE[8]]
        loc=list1+list2
        t=duration[index-81*8]
        y = salinity[8,index-8*81+TIME_WINDOW:index-8*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1
    while ((9*81-PRED_TIME-TIME_WINDOW)<index<9*81):
        index +=1 
        
#O11        
    while ( 9*81<=index<=(10*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[[10,11,12,13,14,15,16,17,18,19,20,9],index-9*81:index-9*81+TIME_WINDOW]
        l=level[[10,11,12,13,14,15,16,17,18,19,20,9],index-9*81:index-9*81+TIME_WINDOW]
        p=pump[[10,11,12,13,14,15,16,17,18,19,20,9],index-9*81:index-9*81+TIME_WINDOW]
        i=infil[[10,11,12,13,14,15,16,17,18,19,20,9],index-9*81:index-9*81+TIME_WINDOW]
        i_s=infil_s[[10,11,12,13,14,15,16,17,18,19,20,9],index-9*81:index-9*81+TIME_WINDOW]
        r = rain[[10,11,12,13,14,15,16,17,18,19,20,9],index-9*81:index-9*81+TIME_WINDOW]
        N9= North[10:21]+[North[9]]
        E9= East[10:21]+[East[9]]
        conn,edge_attr = get_conn_attr(N9,E9)
        list1=NE[10:21]
        list2=[NE[9]]
        loc=list1+list2
        t=duration[index-81*9]
        y = salinity[9,index-9*81+TIME_WINDOW:index-9*81+TIME_WINDOW+PRED_TIME]
        test_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1
    while ((10*81-PRED_TIME-TIME_WINDOW)<index<10*81):
        index +=1 


        
        
    return train_sequences,val_sequences,test_sequences

In [84]:
salinity[[10,11,12,13,14,15,16,17,18,19,20,3],250:250-3*81+TIME_WINDOW]

array([], shape=(12, 0), dtype=float64)

In [99]:
train_sequences,val_sequences, test_sequences = create_dataset(salinity,level,pump,infil,infil_s,rain,NE,duration)
print(len(train_sequences))
print(len(test_sequences))
print(len(val_sequences))
print(train_sequences[122][10])

12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
276
92
92
[1.56 1.56 1.54 1.51 1.53 1.53 1.52 1.5  1.48 1.47 1.44 1.42 1.4  1.38
 1.35 1.33 1.31 1.27]


## 1.3 Create datamodule and dataloader

In [150]:
from torch.utils.data import Dataset, DataLoader
class WellDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self,index):
        sal,lev,pum,inf,inf_s,rai,loca,dur,conn,edge_attr, pred_y = self.sequences[index]
        return dict(
            sal_sequence = torch.tensor(sal),
            lev_sequence = torch.tensor(lev),
            pum_sequence = torch.tensor(pum),
            inf_sequence = torch.tensor(inf),
            inf_s_sequence = torch.tensor(inf_s),
            rai_sequence = torch.tensor(rai),
            loc_sequence = torch.tensor(loca),
            dur_sequence = torch.tensor(dur),
            con_sequence = torch.tensor(conn),
            edg_sequence = torch.tensor(edge_attr),
            
            label = torch.tensor(pred_y).float()   
        )

    

class WellDataModule(pl.LightningDataModule):
    
    def __init__(
        self, train_sequences, val_sequences, test_sequences, batch_size=45
    ):
        super().__init__()
        self.train_sequences = train_sequences
        self.val_sequences = val_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size
        
    def setup(self):
        self.train_dataset = WellDataset(self.train_sequences)
        self.val_dataset = WellDataset(self.val_sequences)
        self.test_dataset = WellDataset(self.test_sequences)
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle = False
        )
    def val_dataloader(self):
        return DataLoader(
        self.val_dataset,
        batch_size=1,
        shuffle = False
        )
    def test_dataloader(self):
        return DataLoader(
        self.test_dataset,
        batch_size=1,
        shuffle = False
        )

In [151]:
N_EPOCHS=8
BATCH_SIZE= 46
##We set the batchsize within a single ob well data, which is 81-18-18+1=46. 
data_module = WellDataModule(train_sequences,val_sequences, test_sequences,batch_size =BATCH_SIZE )
data_module.setup()

In [14]:
train_dataset =  WellDataset(train_sequences)
for item in train_dataset:
    print(item.keys())
    print(item['sal_sequence'].shape)
    break

dict_keys(['sal_sequence', 'lev_sequence', 'pum_sequence', 'inf_sequence', 'inf_s_sequence', 'rai_sequence', 'loc_sequence', 'dur_sequence', 'con_sequence', 'edg_sequence', 'label'])
torch.Size([10, 18])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [152]:
print(train_dataset[1]['con_sequence'])

tensor([[ 0,  1,  3,  4,  5,  6,  8,  8,  9,  9,  9,  9,  9, 10],
        [ 9,  9,  8,  9,  9,  9,  3, 10,  0,  1,  4,  5,  6,  8]])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


# 2.Model

### I set sal_em=8,lev_em=8,pum_em=8,inf_em=8,inf_s_em=8,rai_em=8, loc_em=8 dur_em=8. Then node_h=64, edge_h=1. For each feature, the embed-form dimension should be 46 \*12 \*18 \*8

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean, scatter_add
from torch.utils.data import Dataset, DataLoader

In [159]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean, scatter_add
from torch.utils.data import Dataset, DataLoader
class RNNEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True)

    def forward(self, x, h0, c0):
        # Set initial hidden and cell states
        # Forward propagate LSTM
        out, (h_n, c_n) = self.lstm(x, (h0, c0))  
        # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        return h_n, c_n
class RNNDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True)
        self.lin = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()

    def forward(self, x, h_0, c_0):
        # Forward propagate LSTM
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = self.lin(out)
        out = self.relu(out)
        # Decode the hidden state of the last time step
        return out, h_n, c_n
class WellModel(nn.Module):
    def __init__(self, sal_em, lev_em, pum_em, inf_em, inf_s_em,rai_em, loc_em,dur_em, rnn_h, rnn_l, gnn_h):
        super(WellModel, self).__init__()
        self.sal_em = sal_em
        self.lev_em=lev_em
        self.pum_em=pum_em
        self.inf_em=inf_em
        self.inf_s_em=inf_s_em
        self.rai_em=rai_em
        self.loc_em=loc_em
        self.dur_em=dur_em
        self.node_h=64
        self.edge_h=2
        self.well_num=12
      
        self.rnn_h=rnn_h
        self.gnn_h=gnn_h
        self.rnn_l=rnn_l

        self.embed = nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        print("embed finished",self.embed)
        
        self.sal_embed = nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.lev_embed = nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.pum_embed =nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.inf_embed =nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.inf_s_embed =nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.rai_embed =nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.loc_embed =nn.Sequential(
          nn.Linear(2,8),
          nn.ReLU()
        )
        self.dur_embed =nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        '''
        self.sal_embed = Seq(Lin(1, 8), ReLU()) 
        self.lev_embed = Seq(Lin(1, 8), ReLU())
        self.pum_embed = Seq(Lin(1, 8), ReLU())
        self.inf_embed = Seq(Lin(1, 8), ReLU())
        self.inf_s_embed = Seq(Lin(1, 8), ReLU())
        self.rai_embed = Seq(Lin(1, 8), ReLU())
        self.loc_embed = Seq(Lin(2, 8), ReLU())
        self.dur_embed = Seq(Lin(1, 8), ReLU())'''
                                        ## set sal_em=8,lev_em=8,pum_em=8,inf_em=8,inf_s_em=8,rai_em=8, loc_em=8,dur_em=8 then ,node_h=64
        self.well_gnn = WellGNN(64, 1, gnn_h) # need to do the final: my edge 
                                                             #feature is 1, node feature is the sum of ems,
        self.encoder = RNNEncoder(input_size=gnn_h,
                                  hidden_size=rnn_h,
                                  num_layers=rnn_l)
        self.decoder_embed =nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )
        self.decoder = RNNDecoder(input_size=8 + sal_em,
                                  hidden_size=rnn_h,
                                  num_layers=rnn_l)
    def batchInput(self, x, edge_w, edge_conn):
        well = x.shape[1]
        x = x.reshape(-1, x.shape[-1])
        edge_w = edge_w.reshape(-1, edge_w.shape[-1])
        for i in range(edge_conn.size(0)):
            edge_conn[i, :] = torch.add(edge_conn[i, :], i * well_num)
        edge_conn = edge_conn.transpose(0, 1)  #need to adjust 
        edge_conn = edge_conn.reshape(2, -1)
        return x, edge_w, edge_conn
    
    def forward(self, sequences_data):
        #sta_aqi, sta_conn, sta_poi, sta_w, sta_wea, sta_for, _ = city_data
        well_sal,well_lev, well_pum, well_inf, well_inf_s, rain_data, well_loc, well_dur, well_conn, well_w, _ = sequences_data 
        well_num = 12
        print('this is well sal',well_sal.shape)
        well_a = well_sal.unsqueeze(dim=-1).to(torch.float32)
        print('this is well_a unsqueeze',type(well_a))
        well_a = self.embed(well_a)   # 46*12*18* em_dim
        print('this is well sal.shape',well_a.shape)
        
        well_b = well_lev.unsqueeze(dim=-1).to(torch.float32)
        well_b = self.lev_embed(well_b)      
        print('well_b',well_b.shape)
        well_c = well_pum.unsqueeze(dim=-1).to(torch.float32)
        well_c = self.pum_embed(well_c)
        print('well_c',well_c.shape)
        well_d = well_inf.unsqueeze(dim=-1).to(torch.float32)
        well_d = self.inf_embed(well_d)
        print('well_d',well_d.shape)
        well_e = well_inf_s.unsqueeze(dim=-1).to(torch.float32)
        well_e = self.inf_s_embed(well_e)
        print('well_e',well_e.shape)
        well_f = rain_data.unsqueeze(dim=-1).to(torch.float32)
        well_f = self.rai_embed(well_f)
        print('well_f',well_f.shape)
        print("this is well loc",well_loc.shape)
        well_g = well_loc.unsqueeze(dim=-2).to(torch.float32)
        well_g = self.loc_embed(well_g)
        well_g = well_g.unsqueeze(dim=-3).repeat(1,well_num,18,1)
        print('well_g',well_g.shape)       
        
        well_h = well_dur.unsqueeze(dim=-1).to(torch.float32)
        well_h = self.dur_embed(well_h) 
        well_h = well_h.unsqueeze(dim=-2)
        well_h = well_h.unsqueeze(dim=-3).repeat(1,well_num,18,1)
        print('well_h',well_h.shape)
        x = torch.cat([well_a,well_b,well_c,well_d,well_e,well_f,well_g,well_h],dim=-1)
        x = x.transpose(1, 2) # (batch, well_num, sequence length, emb) to (batch, sequence, well_num, emb)
        x = x.reshape(-1,x.shape[-2],x.shape[-1]) # to (nodes, well_num, emb)
        well_conn = well_conn.repeat(18, 1, 1).to(torch.float32) # I did not transpose
        well_w = well_w.reshape(-1, well_w.shape[-2], well_w.shape[-1]).to(torch.float32) # the first dim batch*
        print('This is x.shape,well_conn.shape,well_w.shape',x.shape,well_conn.shape,well_w.shape)
        x, well_weight, well_conn = self.batchInput(x, well_w, well_conn)
        x = self.well_gnn(x, well_conn, well_weight, well_num)
        x = x.reshape(-1, 18, well_num, x.shape[-1]).transpose(1, 2)
        x = x.reshape(-1, 18, x.shape[-1])
        
        h0 = torch.randn(self.rnn_l, sta_x.size(0), self.rnn_h)
        c0 = torch.randn(self.rnn_l, sta_x.size(0), self.rnn_h)
        h_x, c_x = self.encoder(x, h0, c0)

        outputs = torch.zeros((x.size(0), 18, 1)).to(device)
        sal = well_sal[:,:,-1].reshape(-1,1)
        well_loc_for = well_loc.repeat(well_num, 1,1)

        for i in range(18):
            sal_em = self.decoder_embed(sal)
            inputs = torch.cat((sal_em, well_loc_for[:, i]), dim=-1) # well_loc em = 4
            inputs = inputs.unsqueeze(dim=1)
            output, h_x, c_x = self.decoder(inputs, h_x, c_x)
            output = output.reshape(-1, 1)
            outputs[:, i] = output
            sal = output            
        outputs = outputs.reshape(-1, well_num, well_for.size(1))

        return outputs
class WellGNN(torch.nn.Module):
    def __init__(self, node_h, edge_h, gnn_h):
        super(WellGNN, self).__init__()
        self.node_mlp_1 =nn.Sequential(
          nn.Linear(2 * node_h + edge_h, gnn_h),
          nn.ReLU(inplace=True)
        )

        self.node_mlp_2 =nn.Sequential(
          nn.Linear(node_h + gnn_h, gnn_h),
          nn.ReLU(inplace=True)
        )
      

    def forward(self, x, edge_index, edge_attr,sta_num):
        # x: [N, F_x], where N is the number of nodes.
        # edge_index: [2, E] with max entry N - 1.
        # edge_attr: [E, F_e]
        #u = u.reshape(-1, u.shape[-1])
        #u = u.repeat(sta_num, 1)
        row, col = edge_index
        row=row.to(torch.float32)
        col = col.to(torch.float32)

        out = torch.cat([x[row], x[col], edge_attr], dim=1)
        out = self.node_mlp_1(out)
        out = scatter_mean(out, col, dim=0, dim_size=x.size(0))
        out = torch.cat([x, out], dim=1) #delete u
        return self.node_mlp_2(out)

In [86]:
from torch import nn
sal_embed = nn.Sequential(
          nn.Linear(1,8),
          nn.ReLU()
        )

a = torch.rand(46,12,18,1)
a = sal_embed(a)
print(a.shape)

torch.Size([46, 12, 18, 8])


In [160]:
sal_em=8,
lev_em=8,
pum_em=8,
inf_em=8,
inf_s_em=8,
rai_em=8,
loc_em=8,
dur_em=8,
node_h=64,
edge_h=1,
well_num=12,
batch_size = 46,
rnn_h=64,
gnn_h=32,
rnn_l=1,
lr=0.001,
epoch=10
wells_model = WellModel(8, 8, 8, 8, 8,8, 8,8, 64, 1, 32)
model_num = sum(p.numel() for p in wells_model.parameters()
                      if p.requires_grad)
print('model:', 'Trainable,', model_num)
criterion = nn.MSELoss()
params = wells_model.parameters()
optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0.001)

val_loss_min = np.inf

embed finished Sequential(
  (0): Linear(in_features=1, out_features=8, bias=True)
  (1): ReLU()
)
model: Trainable, 53577


In [161]:
well_num=12
Epoch=1
batch_size=46
data_module = WellDataModule(train_sequences,val_sequences, test_sequences,batch_size)
data_module.setup()
for epoch in range(Epoch):
    #for i ,(well_sal,well_lev, well_pum, well_inf, well_inf_s, rain_data, 
    #        well_loc, well_dur, well_conn, well_w,pred_y) in enumerate(data_module.train_dataloader()):
    for i,j in enumerate(data_module.train_dataloader()):
      print(j.keys())
      well_sal = j['sal_sequence']   
      well_lev = j['lev_sequence']
      well_pum = j['pum_sequence']
      well_inf = j['inf_sequence'] 
      well_inf_s = j['inf_s_sequence'] 
      rain_data = j['rai_sequence']
      well_loc = j['loc_sequence']  
      well_dur = j['dur_sequence']
      well_conn = j['con_sequence']
      well_w = j['edg_sequence']
      pred_y = j['label']

      sequences = [well_sal,well_lev, well_pum, well_inf, well_inf_s, rain_data, 
                well_loc, well_dur, well_conn, well_w,pred_y]
      outputs = wells_model(sequences)
      loss = criterion(outputs, pred_y ) 
      break
      


      wells_model.zero_grad()

      break

dict_keys(['sal_sequence', 'lev_sequence', 'pum_sequence', 'inf_sequence', 'inf_s_sequence', 'rai_sequence', 'loc_sequence', 'dur_sequence', 'con_sequence', 'edg_sequence', 'label'])
this is well sal torch.Size([46, 12, 18])
this is well_a unsqueeze <class 'torch.Tensor'>
this is well sal.shape torch.Size([46, 12, 18, 8])
well_b torch.Size([46, 12, 18, 8])
well_c torch.Size([46, 12, 18, 8])
well_d torch.Size([46, 12, 18, 8])
well_e torch.Size([46, 12, 18, 8])
well_f torch.Size([46, 12, 18, 8])
this is well loc torch.Size([46, 2])
well_g torch.Size([46, 12, 18, 8])
well_h torch.Size([46, 12, 18, 8])
This is x.shape,well_conn.shape,well_w.shape torch.Size([828, 12, 64]) torch.Size([828, 2, 16]) torch.Size([828, 16, 1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


IndexError: ignored